# Legyen Ön is milliomos

A TV-ben nagyon nagy népszerűségnek örvendenek a kvíz játékok, ahol a játékosok lexikális tudását (és szerencséjét) mérik fel. A Raspberry Pi segítségével te is készíthetsz egy otthoni izgalmas kvíz játékot.

Ebben a projektben a Legyen Ön is milliomos vetélkedő egyszerűsített mását készítjük el. Országok fővárosait kell majd kitalálni két opcióból. A két opcióhoz tartozik két gomb is, amivel jelezhetjük melyiket szeretnénk választani. Azt, hogy helyesen választottunk-e LED-ekkel jelezzük, a piros jelenti a rossz, a kék a jó választ. 

## Mit fogsz készíteni?

Kettő különböző színű LED diódából és kettő nyomógombból álló rendszert készítesz, ahol a nyomógomb lenyomásával jelezheted, hogy melyik a helyes válasz, majd a LED kijelzi, hogy a kiválasztott opció helyes-e (kék LED világít) vagy sem (piros LED világít). 

## Mit tanulsz meg?

A reflex verseny elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy egyszerű áramkört összerakni egy breadboard, LEDek, ellenállás, gombok és drótok segítségével
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal
* Hogyan tárolj információt dictionary-ben
* Hogyan írj függvényeket
* Hogyan állapítsd meg, melyik gombot nyomták le
* Hogyan válassz random elemeket listákból

## A projekt részletekre bontása

* Inicializálni a Button és LED objektumokat
* Dictionarybe menteni a kérdéseket
* Függvényeket készíteni a kérdések és válaszok megjelenítéséhez
* Függvényt írni a gomb lenyomásának érzékelésére és a LED kapcsoláshoz
* Végtelen ciklusban megjeleníteni a kérdéseket és várni a választ

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) 2db nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) 2db LED (különböző színű), $I_{max}$ = 20 mA-es: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/tht-led-diodak-5mm_112898/?s_order=desc&search=led&s_field=1000011)

e) 2db 220 - 560 Ohm közötti [ellenállás](https://www.tme.eu/hu/katalog/tht-metal-film-ellenallasok-0-6w_100289/?s_order=desc&search=ellenallas&s_field=1000011)

f) 1db [Breadboard](https://www.tme.eu/hu/katalog/muhelyfelszereles_112607/?s_order=desc&search=breadboard&s_field=1000011)

## A kapcsolási rajz

<img src="prog06_circuit.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t.

1) A nyomógombok lábait illesszük a breadbord mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy oszlopba kerülő lábai csak a lenyomás következtében legyenek összekötve. Mindezt egy műszerrel leellenőrízhetjük.

2) Mindezt ismételjük meg egy másik gombbal legalább 15 sorral eltolva az előző gombhoz képest.

3) Helyezzük az egyik LED lábait két különboző sorba, az egyik nyomógombhoz közelebb. A katódját (negatív láb) kössük a földeléshez míg az anódjának (pozitív láb) sorába kössük be az ellenállásunk egyik lábát, míg a másikat egy üresen álló sorba tegyük. A LED anódjának és katódjának megállapítására használhatunk egy multimétert. Csatlakoztasd a fekete mérőzsinórt a COM (föld) és a piros mérőzsinórt a VΩmA jelzésű hüvelybe. Forgasd el a méréshatárváltó kapcsolót a folytonosság mérés állapotba. Érintsd a két zsínórt a két lábhoz és ha a LED halványan pislákol, akkor az a láb amelyikhez a fekete zsinórt érintetted a katód, a másik az anód. Ha nem pislákol akkor cseréld meg a zsinórok és a lábak érintkezését, hogy világítson.

4) Ismételjük meg az előző pontot a másik LED-del is a másik nyomógombhoz közelebb.

4) Nevezzük ki a breadboard oldalsó oszlopainak egyikét a közös földelésnek (az ábrán a fekete drótok képviselik). Ide kössük be a nyomógombok egyik lábát illetve a LED-ek katódját. Ugyanebbe az oszlopba kössük be a Raspberry Pi egyik **GND** jelölésű tüskéjét is egy jumper drót segítségével.

5) A nyomógombok eddig szabadon lévő lábait egy jumperrel kössük össze a **2**-es és **3**-as GPIO tüskével.

6) Az kék LED ellenállásának szabadon levő lábát egy jumperrel kössük össze a **10**-es GPIO tüskével, míg a pirosét a **9**-es GPIO tüskével.

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```led_quiz.py``` név alatt.

### A LED kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, hogyan viselkedjen amikor a gombot megnyomják.

Először betöltjük a ```gpiozero``` modulból a ```Button``` és ```LED``` objektumokat amik lehetővé teszik, hogy a Raspi kommunikáljon a gombbal és a LED-del a **GPIO** tüskéken keresztül. Ezek mellett beolvassuk még a ```sleep``` függvényt is, amit a python várakoztatására használunk majd, illetve a ```random``` csomagot, amivel a kérdések és a válasz opciók megjelenését teszzük majd véletlenszerűvé.

```led_quiz.py```:

In [6]:
from gpiozero import Button, LED
from time import sleep
import random

Az objektum beolvasása után inicializálunk kettő ```LED``` objektumot, amit ```red```-nek és ```blue```-nak nevezünk el. Egyben megmondjuk, hogy azokat a *10*-es és a *9*-es számú **GPIO** tüskére kötöttük.  

```led_quiz.py```:

In [ ]:
from gpiozero import Button, LED
from time import sleep
import random

blue = LED(9)
red = LED(10)

Ezután, hogy tesztelhessük, működik-e az áramkörünk LED-es része, egészítsük ki a kódot a következő sorokkal, ami villogtatja a LED-eket:

```led_quiz.py```:

In [ ]:
from gpiozero import LED, Button
from time import sleep
import random

blue = LED(9)
red = LED(10)

blue.blink()
red.blink()
sleep(5)

Mentsük el a fájlt és futtasuk le a kódot a terminálban a ```python led_quiz.py``` paranccsal. Ha minden rendben van, a két LED villog 5 másodpercig majd kialszik.

### A nyomógombok tesztelése

Inicializáljunk most két nyomógombot, ```yes``` és ```no```, amiket a *2*-es és a *3*-as GPIO tüskékre kötünk. 

```led_quiz.py```:

In [ ]:
from gpiozero import LED, Button
from time import sleep
import random

blue = LED(9)
red = LED(10)

yes = Button(2)
no  = Button(3)

Ezután írunk egy ```pressed``` nevű függvényt, aminek az első argumentuma egy gomb objektum a második pedig egy segéd argumentum, aminek az értéke 1 ha a ```yes``` gombot nyomtuk meg illetve 2 ha a ```no```-t. A függvényen belül egy ```if``` szerkezettel kivizsgáljuk, hogy mi az értéke ennek a második argumentumnak, és ha 1 akkor kinyomtatjuk, hogy ```You got it right!```, ha 2 akkor pedig ```Wroooong!```. Majd a végső kódban ezeket a kiírásokat a megfelelő LED bekapcsolása fogja leváltani. Miután kész a függvény, azt hozzá kell rendelni a nyomógombok ```when_pressed``` metódusához, hogy a lenyomásra ez a függvény aktiválódjon. 

Végül, írunk egy üres végtelen ```while``` ciklust, hogy le tudjuk tesztelni a gombjaink működését. 

```led_quiz.py```:

In [ ]:
from gpiozero import LED, Button
from time import sleep
import random

blue = LED(9)
red = LED(10)

yes = Button(2)
no  = Button(3)

def pressed(button, pos = 0):
    if pos == 1:
        print('You got it right!')
    else:
        print('Wroooong!')
    
yes.when_pressed = pressed(yes, pos = 1)
no.when_pressed  = pressed(no, pos = 2)

while True:
    pass

Ha a LED-ek vagy a gombok nem követik a programban lekódolt viselkedést, ellenőrízzük az áramkör kapcsolását, hogy minden jól lett-e bekötve. 

### Kvíz kérdésekhez függvények definiálása

Hogy a fő programrészt rövidebbre vehessük, írunk néhány segédfüggvényt, amik lehetővé teszik a kérdések és a hozzájuk tartozó lehetséges válaszok megjelenítését. 

Először létrehozunk egy dictionary-t, aminek a kulcsai az ország nevek, az értékei pedig a kulcsokhoz tartozó fővárosok. Ebből az adatbázisból fogunk véletlenszerűen kérdéseket formálni. 

In [4]:
qa = {
    'Törökország': 'Ankara',
    'Albánia'    : 'Tirana',
    'Argentína'  : 'Buenos Aires',
    'Ausztria'   : 'Bécs',
    'Görögország': 'Athén',
    'Szerbia'    : 'Belgrád',
    'Németország': 'Berlin',
    'Ausztrália' : 'Canberra',
    'Írország'   : 'Dublin',
    'Norvégia'   : 'Oslo',
}

A következő lépésben függvényt írunk, ```question```, a kérdés megszerkesztésére. Egy argumentumot, ```country``` fogad el, ami az ország neve. Ennek a függvénynek nem más a feladata, mint behelyettesíteni az ország nevet az előre megszerkesztett kérdés mintába. 

In [2]:
def question(country):
    txt = f'Mi {country} fővárosa?'
    return txt

question('Kína')

'Mi Kína fővárosa?'

Egy példán keresztül, ahol a bemenő paraméter 'Kína', láthatjuk is, hogy a függvény visszaadott értéke egy string a következő szöveggel: 'Mi Kína fővárosa'.

A következő függvény, ```options```, egy kicsit bonyolultabb. Ennek a célja, hogy megjelenítsen véletlenszerű sorrendben két választ, amelyből az egyik a helyes, míg a másik egy véletlenszerűen kiválasztott főváros. Ahhoz, hogy a nyogombokról meg tudjuk mondani, hogy a jó válaszra utalnak-e, tudnunk kell, hogy a helyes válasz az első vagy a második helyen jelenik meg. Ennek rögzítésére hozzuk létre a ```right_answer``` változót, amit kezdetben 0-nak veszünk, de működés közben az értéke vagy 1 vagy 2.

Két bemenő argumentuma van, ```city```, ami a helyes válasz és az ```opt``` ami egy lista a lehetséges főváros nevekkel. A ```right_answer``` változót globálissá tesszük, hogy a függvényen belül meg tudjuk változtatni az értékét. Ezután véletlenszerűen kiválasztunk egy fővárost az ```opt``` listából a ```random.choice(opt)``` paranccsal és elmentjük azt az ```a``` lista változóba. Ehhez a listához, második helyre hozzáadjuk a helyes fővárost, ```a.append(city)```. Mivel idáig, minden függvénymeghíváskor a helyes válasz a második helyen lenne, megkeverjük véletlenszerűen a két elemet a ```random.shuffle(a)``` paranccsal. Vegyük észre, hogy ez a parancs helyben megváltoztatja ```a``` szerkezetét, nem kellett hozzárendelnünk ```a```-hoz. Ezek után a ```txt``` változóba elmentjük a két válasz lehetőséget. Végül módosítjuk a ```right_answer``` értéket annak megfelelően, hogy az 1)-es vagy a 2)-es lehetőség a jó válasz. Az ```a.index(city)``` metódussal megkeressük, hogy a helyes válasznak mi az indexe a listán belül (0 vagy 1), majd hozzáadunk 1-et, hogy 1-re vagy 2-re módosuljon.  

In [11]:
right_answer = 0

def options(city, opt):
    global right_answer
    a = [random.choice(opt)]
    a.append(city)
    random.shuffle(a)
    txt = f'1) {a[0]}      2) {a[1]}'
    right_answer = a.index(city) + 1
    return txt

country = random.choice(list(qa.keys()))
city    = qa[country]

print(question(country))
print(options(city, list(qa.values())))
print(f'right_anser = {right_answer}')

Mi Norvégia fővárosa?
1) Oslo      2) Ankara
right_anser = 1


Egy példán keresztül a fenti tömben be is mutatjuk, hogy működik a kérdés és a válasz opciók megjelenítése. 

* Véletlenszerűen választunk egy országnevet és azt elmentjük a ```country``` változóba. 
* Megkeressük a hozzá tartozó fővárost és a ```city```-hez rendeljük.
* Kinyomtatjuk a kérdést a ```question(country)``` függvénnyel.
* Kinyomtatjuk a lehetséges válaszokat az ```options(city, list(qa.values()))``` függvénnyel.
* Végül kinyomtatjuk a helyes válasz sorszámát is.

Mostmár meg van minden elemünk, csak össze kell rakni a teljes programot.

### A kvíz elkészítése

Eljutottunk arra a pontra, amikor már elkészíthetjük a teljes kvíz játékunkat. Elsőnek definiálunk egy új globális változót, ```press = False```, aminek a feladata, hogy a kódon keresztül számon tartsa lenyomták-e az egyik gombot vagy sem. Alap állapotban értéke ```False```, jelezve, hogy a gombot nem nyomták meg. 

A következő módosítás a ```pressed(button, pos=0)``` függvényben látható. A gomb nyomására most már nem a terminálra nyomtatunk ki, hanem a LED-eket kapcsoljuk. Elsőnek, globálissá nyilvánítjuk a ```press``` és a ```right_answer``` változókat, hogy úgy módosíthassuk őket, hogy a fő program rész is látja azt. Ezután az ```if pos == right_answer:``` szerkezettel megvizsgáljuk, hogy a lenyomott gombhoz tartozó szám, ```pos```, megegyezik-e a helyes válasz sorszámával, ```right_answer```. Ha igen, akkor a kék LED-et felkapcsoljuk 3 másodperc erejéig, ha nem akkor a piros LED-et kapcsoljuk fel 3 másodpercig. Végül a ```press``` globális változót visszaállitjuk hamisra. Evvel jelezzük, hogy jöhet a következő kérdés (ahogy a következő paragrafusban azt látni fogjuk). 

Végül a végtelen ```while``` ciklust kell megírnunk, amiben újra és újra teszünk fel kvíz kérdéseket. Itt először megvizsgáljuk, hogy a ```press``` értéke hamis-e. Ezt az ```if not press:``` kifejezéssel tesszük. Ha a ```press``` hamis, akkor jelzi, hogy lehet kérdést feltenni, ha igaz, akkor viszont nem megy bele a szerkezetbe. Ez fontos, hogy két egymást követő ```while``` ciklus ne tegyen fel azonnal egy új kérdést, hanem csak azután, hogy a gombnyomással válaszoltunk. Így az első lépésünk az ```if``` szerkezetben, hogy a ```press``` változót átírjuk igazra. Azután pedig kiírathatjuk a képernyőre a kérdést és a válaszlehetőségeket. Ezeket a ```print('='*10)``` paranccsal választjuk el, ami 10 egyenlőségjelet nyomtat ki egy sorba. Ha egy stringet megszorzunk egy számmal, akkor a végeredményünk az eredeti string megismételve a megadott számszor. 

Evvel a kódunk végére értünk. Mentsük el és teszteljük le.

```led_quiz.py```:

In [ ]:
from gpiozero import Button, LED
from time import sleep
import random

blue = LED(9)
red = LED(10)

yes = Button(2)
no  = Button(3)

qa = {
    'Törökország': 'Ankara',
    'Albánia'    : 'Tirana',
    'Argentína'  : 'Buenos Aires',
    'Ausztria'   : 'Bécs',
    'Görögország': 'Athén',
    'Szerbia'    : 'Belgrád',
    'Németország': 'Berlin',
    'Ausztrália' : 'Canberra',
    'Írország'   : 'Dublin',
    'Norvégia'   : 'Oslo',
}

press = False
right_answer = 0


def question(country):
    txt = f'Mi {country} fővárosa?'
    return txt

def options(city, opt):
    global right_answer
    a = [random.choice(opt)]
    a.append(city)
    random.shuffle(a)
    txt = f'1) {a[0]}      2) {a[1]}'
    right_answer = a.index(city) + 1
    return txt

def pressed(button, pos = 0):
    global press, right_answer
    if pos == right_answer:
        blue.on()
        sleep(3)
        blue.off()
    else:
        red.on()
        sleep(3)
        red.off()
    press = False

yes.when_pressed = pressed(yes, pos = 1)
no.when_pressed  = pressed(no, pos = 2)

while True:
    if not press:
        press = True
        country = random.choice(list(qa.keys()))
        city    = qa[country]
        print(question(country))
        print('='*10)
        print(options(city, list(qa.values())))
        print('='*10)

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```led_quiz.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```led_quiz.py```-t elmentettük. Ott begépelve a ```python led_quiz.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor kérdések jelennek meg a képernyőn és várnak amíg a gomb lenyomásával válaszolunk, majd a LED-ek jelzik, hogy jó volt-e a válasz. Ezután jön az újabb kérdés.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Számoljuk, hogy hány helyes választ adtak.
* Hangszórót/fejhallgatót kapcsolva a Raspberry-re bípeltessük a gépet minden gomb lenyomásnál.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) LED objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#led](https://gpiozero.readthedocs.io/en/stable/api_output.html#led)

3) Button objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_input.html#button](https://gpiozero.readthedocs.io/en/stable/api_input.html#button)

4) while ciklus - [https://realpython.com/python-while-loop/](https://realpython.com/python-while-loop/)